# Cours Introduction à la Recherche d'Information à l'N7

# Indexation, il existe différentes librairies sous Python pour analyser le texte. 
## L'analyse peut consister à extraire les mots simples, jusqu'à l'analyse un peu plus "poussée". Vecteurs de mots pondérés, jusqu'aux embeddings.

## De manière générale le  "text processing" passe par les étapes suivantes
    - tokenizer—scanner, lexer, lexical analyzer
    - vocabulary—lexicon
    - token, term, word, or n-gram—token, symbol, or terminal symbol


## Des exemples de Librairies : NLTK, Sklearn, Gensim

## 1- NLTK

In [ ]:
#NLTK
import nltk

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize #TweetTokenizer
from nltk.util import ngrams
from nltk.probability import FreqDist

from pprint import pprint

#NLTK
#- tokenizer—scanner, lexical analyzer
def extraire_bag_of_words(texte, type_norma):  
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Method-1 : Individual words as separate elements of the list   
    #text split extraire tous les mots, séparateurs, ...
    texte_split=texte.split()
    
    # seuls les mots (tokens) sont extraits) 
    texte = word_tokenize(texte.lower())

    # suppression des stop words
    stop_words = set(stopwords.words('english'))

    #Liste des mots sans les mots vides
    new_tokens = [w for w in texte if not w in stop_words]
    # Liste des mots normalisés (selon Porter)
    if type_norma=='porter':      
        new_tokens = [stemmer.stem(w) for w in texte if not w in stop_words]
        #print("Normalisation selon Porter")
        #print(new_tokens)

    # Liste des mots normalisés (selon Lemmatiseur)
    if type_norma=='lemmatizer':      
        new_tokens = [lemmatizer.lemmatize(w) for w in texte if not w in stop_words]
        #print("Normalisation selon lemmatiseur")
        #print(new_tokens)
    
    return new_tokens

def count_words(tokens):
    # librairie pour le comptage des mots
    from collections import Counter
    bag_of_words = Counter(tokens)
    return bag_of_words




In [ ]:
text_file = 'data/sample.txt'
with open(text_file) as f:
        text=f.read()

##   Extraire les mots simple et lémamtiser avec Porter ou un lemattizar plus sophistiqué (par exemple WordNet)

tokens=extraire_bag_of_words(text,'porter')
#tokens=extraire_bag_of_words(text,'lemmatizer')

print(tokens)

#récuperer la liste des n grammes (récupérer les mots adjenents 2 à deux.)
list(ngrams(tokens,2))

#Le texte peut comporter plusieurs phrases on peut "Split" les phrases (passages) du texte.
print(sent_tokenize(text))

# compter les tokens (construire le bag of words (Mot,count))
#bag_words=count_words(tokens)
#print(bag_words)


In [ ]:
#Uilisation de WordNet our une représentation "conceptuelle"
from nltk.corpus import wordnet

word_ = wordnet.synsets("spectacular")
print (word_[2].definition())


## 2- SKLEARN : Vectorisation des textes avec sklearn

In [ ]:
#NLP avec scikit-learn 
# Libraire pour la vectorization des textes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#operation en deux étapes fit (pour le vocabulaire) puis tranform (pour construire les vecteurs)  
def vectorize_2_steps(text):
    cv = CountVectorizer() 
    #vectorization    
    vectors = cv.transform(text)      
    return vectors, cv.get_feature_names()

#operation fit et transform avec une seule fonction 
def vectorize_1_step(text):
    cv = CountVectorizer() 
    vectors=cv.fit_transform(text)    
    return vectors, cv.get_feature_names()

def simple_vector(text):
    cv = CountVectorizer(stop_words='english',token_pattern=r'\w+')
    vectors = cv.fit_transform(text).todense()  
    return vectors, cv.get_feature_names()

def tf_idf_vector(text):
    t = TfidfVectorizer(stop_words='english', token_pattern=r'\w+')
    vectors = t.fit_transform(text).todense()
    return vectors


### Test sur la vectorisation avec SKlearn

In [ ]:
text_file = 'data/sample.txt'

with open(text_file) as f:    
    text=f.readlines()

# Construire les vecteurs 
#Les fonction CountVectorizer, simple façon de "tokeniser" un texte, 

#vectors,features = vectorize_1_step(text)

# vecteur dense
#vectors,features = simple_vector(text)
#print(vectors.shape)
#print(features)
#print(vectors)
#print(vectors.toarray())  # affichage mieux que print(vectors) si le vectru n'est pas mis en "todense()" dans la fonction

vectors= tf_idf_vector(text)
print(vectors.shape)
print(vectors)



## 3- Librairie GENSIM

### La première partie donne un exemple de "tokenisation" assez simple , on crée un dictionnaire, puis on transfome les mots en id (token2id). On crée un simple dictionnaire à partir d'un texte, d'un fichier ou un répétoire

In [ ]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
import os
# Choisir une option lecture du Texte et la procédure de construction des tockens
            # un simple texte
#Text =["""Global warming is a long-term rise in the average temperature of the Earth s climate system, 
#       an aspect of climate change shown by temperature measurements and by multiple effects of the warming."""]

            ## version simple lectrure ligne par phrase puis split()
#tokens = [[token for token in sentence.split()] for sentence in text]
            ## utilisation du pre_pocessing de gensim
#tokens= [simple_preprocess(remove_stopwords(sentence)) for sentence in Text]
            #avec suprenssion de mots vides
#tokens = [simple_preprocess(remove_stopwords(sentence)) for sentence in Text]

            ##Texte dans un Un fichier
#tokens=[simple_preprocess(sentence) for sentence in open('data/dataN7/text_file1.txt', encoding='utf-8')]

## Plusieurs fichiers dans un répértoire
dir_path= 'data/dataN7'
for file_name in os.listdir(dir_path):
    if ".txt" in file_name:  
        tokens = [simple_preprocess(remove_stopwords(sentence)) for sentence in open(os.path.join(dir_path, file_name), encoding='utf-8')]

    ##Construire le dictionnaire
gensim_dictionary = corpora.Dictionary(tokens)

##Afficher les mots
gensim_dictionary.token2id


### On crée un sac de mots (Id, frequence). On utilise  la méthode  doc2bow, de gensim_dictionary.Chaque mot est traité séparemment, si il existe dans le dictionnaire sa fréquence est incrémentée sinon il est crée avec ne fréquence de 1.

In [ ]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
import os

## on lit les documents à partir d'un fichier (pas besoin de lire plusieurs fois le fichier, je le fais pour que chque cellule puisse être exécutée séparement.)
dir_path= 'data/dataN7'
for file_name in os.listdir(dir_path):
    if ".txt" in file_name:  
        print(file_name)
        tokens = [simple_preprocess(remove_stopwords(sentence)) for sentence in open(os.path.join(dir_path, file_name), encoding='utf-8')]

gensim_dictionary = corpora.Dictionary(tokens)
gensim_corpus = [gensim_dictionary.doc2bow(token, allow_update=True) for token in tokens]

##Afficher les ID ainsi que leeur fréquence)
print(gensim_corpus)

    ## on peut aussi aficher les mots ainsi que leur fréqueence
word_frequencies = [[(gensim_dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

print(word_frequencies)

### On passe à la vectorisation (on crée un bag of words pondéré par tf.idf (a la SMART (modèle vectriel))). On utilise la librairie models de gensim 

In [ ]:
from gensim import models
import numpy as np

    ## Je récupre le coprus que j'ai construit précédemment
    ## J'appelle la méthode TfidfModel de mdels

tfidf = models.TfidfModel(gensim_corpus, smartirs='ltc')

for doc in tfidf[gensim_corpus]:
    print(doc)
    for id, frequency in doc:
        print(gensim_dictionary[id], np.around(frequency, decimals=2))
    
    


# LISI et LDA
## LSI

In [ ]:
api.info("text8")  # return dict with info about "text8" dataset

In [ ]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim.models import LsiModel
import os

#Etape 1 récuperer les documents
## on lit les documents à partir d'un fichier (pas besoin de lire plusieurs fois le fichier, 
#je le fais pour que chque cellule puisse être exécutée séparement. On récupère les mots lémmatisés)
#dir_path= 'data/dataN7'
#for file_name in os.listdir(dir_path):
#    if ".txt" in file_name:  
#        print(file_name)
#        tokens = [simple_preprocess(remove_stopwords(sentence)) for sentence in open(os.path.join(dir_path, file_name), encoding='utf-8')]

#Prendcre un GROS fichier exemple proposé par Gensim
dataset = api.load("text8")
tokens = [d for d in dataset]

##Afficher les ID ainsi que leur fréquence)
#print(gensim_corpus)


# Step 3: Créer les entrées de LSI : Dictionnaire et le corpus 
        
gensim_dictionary = corpora.Dictionary(tokens)
gensim_corpus = [gensim_dictionary.doc2bow(token, allow_update=True) for token in tokens]

lsi = models.LsiModel(corpus=gensim_corpus, id2word=gensim_dictionary, num_topics=4) 

lsi.show_topics()

## LDA

In [ ]:
## On récupère le corpus et le dictionnaire précédents :  gensim_dictionary  et gensim_corpus
from gensim.models import LdaModel, LdaMulticore

lda_model = LdaMulticore(corpus=gensim_corpus,
                         id2word=gensim_dictionary,
                         random_state=100,
                         num_topics=7,
                         passes=10,
                         chunksize=1000,
                         batch=False,
                         alpha='asymmetric',
                         decay=0.5,
                         offset=64,
                         eta=None,
                         eval_every=0,
                         iterations=100,
                         gamma_threshold=0.001,
                         per_word_topics=True)

# save the model
lda_model.save('lda_model.model')

lda_model.print_topics(-1)


In [ ]:
for c in lda_model[gensim_corpus[5:8]]:
    print("Document Topics      : ", c[0])      # [(Topics, Perc Contrib)]
    print("Word id, Topics      : ", c[1][:3])  # [(Word id, [Topics])]
    print("Phi Values (word id) : ", c[2][:2])  # [(Word id, [(Topic, Phi Value)])]
    print("Word, Topics         : ", [(dct[wd], topic) for wd, topic in c[1][:2]])   # [(Word, [Topics])]
    print("Phi Values (word)    : ", [(dct[wd], topic) for wd, topic in c[2][:2]])  # [(Word, [(Topic, Phi Value)])]
    print("------------------------------------------------------\n")

In [ ]:
## 
#LSI
from gensim.models import LdaModel, LdaMulticore
from gensim.utils import simple_preprocess, lemmatize

#my_dict, my_corpus=get_dict_corpus()
    
# initialize an LSI transformation
my_dict = corpora.Dictionary.load('data/my_dict.dict')
my_corpus = corpora.MmCorpus('data/bow_corpus.mm')

#reécupérer ue représentation d'un document

#print('LDA')
lda_model = models.LdaMulticore(my_corpus,id2word=my_dict,random_state=100,num_topics=5)

#lda_model.print_topics(-1)
lda_model.show_topics()


# lire les fichiers

lda_model[my_corpus]

#for c in lda_model[my_corpus[0:3]]:
#    print("Document Topics      : ", c[0])      # [(Topics, Perc Contrib)]
#    print("Word id, Topics      : ", c[1][:])  # [(Word id, [Topics])]
#    print("Phi Values (word id) : ", c[2][:2])  # [(Word id, [(Topic, Phi Value)])]
#    print("Word, Topics         : ", [(my_dict[wd], topic) for wd, topic in c[1][:2]])   # [(Word, [Topics])]
#    print("Phi Values (word)    : ", [(my_dict[wd], topic) for wd, topic in c[2][:2]])  # [(Word, [(Topic, Phi Value)])]
    #print("------------------------------------------------------\n")


In [ ]:
data=gensim_preprocess_data()
data[1:10]

# Word2Vec Sous GENSIM

## Utiliser des vecteurs (W2VEC) Préentrainés : Word2Vec, Glove et FastText 
### Utilisation de Word2Vec (WOrd2Vec est sur mon ordinateur)

In [ ]:
from gensim.models import KeyedVectors
import os
# load the google word2vec model
filename = '../../data/GoogleNews-vectors-negative300.bin'
if os.path.exists(filename) : 
    #charge le model en mémoire
    model1 = KeyedVectors.load_word2vec_format(filename, binary=True)
else:
    print('le fichier n existe pas')

### Utiisation de Glove (nécessite une conversion vers le format Word2Vec)

In [ ]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

#Utilisation de Glove de Stanford 
#The first step is to convert the GloVe file format to the word2vec file format. 
glove_input_file = '../../data/Glove/glove.6B.100d.txt'
glove_output_file = '../../data/Glove/glove.6B.100d.txt.word2vec'
#convert txt en word2Vec Format
glove2word2vec(glove_input_file, glove_output_file)

# UNE FOIS CONVERTI, On UTILISE LE MOD7LE comme ci-dessous
# load the converted model
filename = '../../data/Glove/glove.6B.100d.txt.word2vec'
glove_model = KeyedVectors.load_word2vec_format(filename, binary=False)

### Gensim propose une api (ci dessous) qui permet de charger le fichier (à prtir de votre ordniteur si il existe dans gensim-data/ ou , le télécharger à partir d'un site extérieur ça peut prendre du temps

In [ ]:
# Gensim propose une api (ci dessous) qui permet de charger le fichier (de votre ordnitauer si il existe dans gensim-data/ 
#ou , le téléchargé à partir d'un site extérieur ça peut prendre du temps
import gensim.downloader as api

# Download the models
#ça pourrait prendre un peu de temps si vous l'avez pas localement dans la librarire de gensim")
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')
word2vec_model300 = api.load('word2vec-google-news-300')
glove_model300 = api.load('glove-wiki-gigaword-300')

## On peut jouer avec les modèles chargés

In [ ]:
#calul de la similarité

# avec Word2Vec
word2vec_model300.most_similar('trump',topn=5)

# avec FastText
fasttext_model300.most_similar('trump',topn=5)

# Avec Glove
glove_model300.most_similar('trump',topn=5)

### Cmme on maniplue des vecteurs , on peut s'amuser à faire ce type de calcul (Vect(king)-Vect(man)+Vect(woman) .... le vecteur résultat est proche du vecteur du mot "Queen".

In [ ]:
# calculate: (king - man) + woman = ?
result = word2vec_model300.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
#result = glove_model.most_similar(positive=['france', 'china'], negative=['paris'], topn=3)
print(result)

In [ ]:

sent_1 = 'Sachin is a cricket player and a opening batsman'.split()
sent_2 = 'Dhoni is a cricket player too He is a batsman and keeper'.split()
sent_3 = 'Anand is a chess player'.split()

# Prepare a dictionary and a corpus.
similarity = word2vec_model300.wmdistance(sent_1, sent_2)
print(f"{similarity:.4f}")


In [ ]:
from gensim.matutils import softcossim
from gensim import corpora

sent_1 = 'Sachin is a cricket player and a opening batsman'.split()
sent_2 = 'Dhoni is a cricket player too He is a batsman and keeper'.split()
sent_3 = 'Anand is a chess player'.split()

# Prepare a dictionary and a corpus.
documents = [sent_1, sent_2, sent_3]
dictionary = corpora.Dictionary(documents)

# Prepare the similarity matrix
similarity_matrix = word2vec_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences into bag-of-words vectors.
sent_1 = dictionary.doc2bow(sent_1)
sent_2 = dictionary.doc2bow(sent_2)
sent_3 = dictionary.doc2bow(sent_3)

# Compute soft cosine similarity
print(softcossim(sent_1, sent_2, similarity_matrix))

print(softcossim(sent_1, sent_3, similarity_matrix))

print(softcossim(sent_2, sent_3, similarity_matrix))


## Creation de son propre word2Vec

In [ ]:
from gensim.models import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api

# Download dataset
dataset = api.load("text8")

data = [d for d in dataset]

# Split the data into 2 parts. Part 2 will be used later to update the model
data_part1 = data[:1000]
data_part2 = data[1000:]

# Train Word2Vec model. Defaults result vector size = 100
model = Word2Vec(data_part1, min_count = 1, workers=cpu_count())

# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
#print(words)

# Save and Load Model
model.save('Text8_W2Vec')
model = Word2Vec.load('Text8_W2Vec')

In [ ]:
# access vector for one word
print(model['sentence'])


## Mise à jour d'un W2Vec (on prend la 2nde partie de notre fichier)

In [ ]:
# Update the model with new data.
model.build_vocab(data_part2, update=True)
model.train(data_part2, total_examples=model.corpus_count, epochs=model.iter)
model['topic']
# array([-0.6482, -0.5468,  1.0688,  0.82  , ... , -0.8411,  0.3974], dtype=float32)

## Autres création d'un W2Vec

In [ ]:
import gensim
from gensim.models import Word2Vec
from multiprocessing import cpu_count
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
import os


dir_path= 'data/dataN7'

for file_name in os.listdir(dir_path):
    if ".txt" in file_name:  
        print(file_name)
        tokens = [simple_preprocess(sentence) for sentence in open(os.path.join(dir_path, file_name), encoding='utf-8')]

#print(tokens)
        

model = Word2Vec(tokens, min_count = 1, workers=cpu_count())

# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
#print(model['sentence'])

# access vector for one word
print(model['earth'])



In [ ]:
#save the model
model.save('data/dataN7/w2vec_model')

In [ ]:
#Save model en ascii 
model.wv.save_word2vec_format('data/dataN7/w2vec_model_ascii', binary=False)


In [ ]:
#Load the model
model = Word2Vec.load('data/dataN7/w2vec_model')

In [ ]:
# summarize vocabulary
words = list(model.wv.vocab)
print(words)

# Construction d'un fichier TF*IDF

In [ ]:
from gensim.utils import simple_preprocess
from smart_open import smart_open
import os
# Create gensim dictionary form a single tet file
dictionary = corpora.Dictionary(simple_preprocess(line, deacc=True) for line in open('data/dataN7/text_file1.txt', encoding='utf-8'))

dictionary.token2id

In [ ]:
min_count = 0
size = 50
window = 3
corpus=[]

#construire son propre w2Vec
for sentence in texte:
    corpus.append(sentence.split())
    model = Word2Vec(corpus, min_count=min_count, size=size, window=window)

#pprint(model.most_similar('the'))
# appeler un W2Vec 
#print("début")
#model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

#pprint("fin")
#pprint(model.most_similar('hello'))
#pprint("fin2")                                                     
#tokens_s=[]
#for token in texte:
    # tokens_s.append(stemmer.stem(token))
    # tokens_s.append(lemmatizer.lemmatize(token))